In [1]:
data_dir = '/home/ubuntu/data/iceberg'

import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers.merge import Add, Concatenate
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
import cv2
import keras
import os
from keras.callbacks import TensorBoard

from keras import __version__
print(__version__)


Using TensorFlow backend.


2.1.1


In [2]:
np.random.seed(1234)

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

#         imgs.append(np.dstack((band_1, band_2, band_3)))
        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [3]:
df_train = pd.read_json(os.path.join(data_dir, 'train.json'))
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xinc = df_train.inc_angle[idx_tr[0]]

In [4]:
Xtrain[1,:,:,1].max()

0.72976856454753891

In [5]:
# Create a train and validation split, 75% of data used in training
from sklearn.model_selection import train_test_split

X_train, X_valid, X_inc_train, X_inc_valid, Y_train, Y_valid = train_test_split(Xtrain,
                                    Xinc, Ytrain, random_state=777, train_size=0.9, test_size=0.1)

X_train.shape, X_valid.shape, X_inc_train.shape, X_inc_valid.shape, Y_train.shape, Y_valid.shape

((1323, 75, 75, 3), (148, 75, 75, 3), (1323,), (148,), (1323,), (148,))

In [6]:
X_train = get_more_images(X_train)
X_inc_train = np.concatenate((X_inc_train,X_inc_train,X_inc_train))
Y_train = np.concatenate((Y_train,Y_train,Y_train))

X_train.shape, X_inc_train.shape, Y_train.shape

((3969, 75, 75, 3), (3969,), (3969,))

In [7]:
batch_size = 256
        
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/tensorboardlogs/', histogram_freq=0, write_graph=True, write_images=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')

In [8]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [10]:

model.fit([X_train, X_inc_train], to_categorical(Y_train),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 3969 samples, validate on 148 samples
Epoch 1/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.0892 - acc: 0.9680 - val_loss: 0.3063 - val_acc: 0.9189
Epoch 2/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.0868 - acc: 0.9660 - val_loss: 0.3030 - val_acc: 0.9257
Epoch 3/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.0813 - acc: 0.9746 - val_loss: 0.3066 - val_acc: 0.9324
Epoch 4/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.0794 - acc: 0.9729 - val_loss: 0.3125 - val_acc: 0.9257
Epoch 5/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.0812 - acc: 0.9694 - val_loss: 0.3245 - val_acc: 0.9392
Epoch 6/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.0708 - acc: 0.9730 - val_loss: 0.3197 - val_acc: 0.9426
Epoch 7/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.0776 - acc: 0.9728 - val_loss: 0.3066 - val_acc: 0.9392
Epoch 8/50
3969

In [11]:
model.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-data2_94_acc.h5')

In [38]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])


1471/1471 [==============================] - 1s 647us/step
Train score: 0.0566026721131
Train accuracy: 0.987763426241


In [13]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('/home/ubuntu/data/iceberg/results/final_preds/cnn_model1_aug_data2' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  2.231949e-03
1  4023181e  9.414673e-01
2  b20200e4  9.350348e-01
3  e7f018bb  9.999995e-01
4  4371c8c3  1.864136e-01
5  a8d9b1fd  9.991841e-01
6  29e7727e  3.206558e-03
7  92a51ffb  9.999993e-01
8  c769ac97  3.583631e-12
9  aee0547d  8.747134e-11


In [21]:
# pseudo labelling using the above model

In [14]:
# this is the correct way round with [:,1]
idx_pred_1 = (np.where(pred_test[:,1]>0.95))
idx_pred_0 = (np.where(pred_test[:,1]<0.05))

In [15]:
Xtest_inc.shape, X_inc_train.shape

((8424,), (3969,))

In [16]:
Xtest_inc = np.expand_dims(Xtest_inc, axis=1)
X_inc_train = np.expand_dims(X_inc_train, axis=1)

In [17]:
Xtrain_pl = np.concatenate((X_train, Xtest[idx_pred_1[0],...], Xtest[idx_pred_0[0],...]))
Ytrain_pl = np.concatenate((Y_train, np.ones(idx_pred_1[0].shape[0]), np.zeros(idx_pred_0[0].shape[0])))
Xinc_pl = np.concatenate((X_inc_train, Xtest_inc[idx_pred_1[0],...], Xtest_inc[idx_pred_0[0],...]))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((10203, 75, 75, 3), (10203,), (10203, 1))

In [18]:
Xinc_pl = np.squeeze(Xinc_pl)

In [19]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model2 = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model2.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 10203 samples, validate on 148 samples
Epoch 1/50
10203/10203 [==============================] - 16s 2ms/step - loss: 0.0550 - acc: 0.9799 - val_loss: 0.2761 - val_acc: 0.9291
Epoch 2/50
10203/10203 [==============================] - 16s 2ms/step - loss: 0.0534 - acc: 0.9802 - val_loss: 0.2824 - val_acc: 0.9223
Epoch 3/50
10203/10203 [==============================] - 16s 2ms/step - loss: 0.0514 - acc: 0.9810 - val_loss: 0.2703 - val_acc: 0.9223
Epoch 4/50
10203/10203 [==============================] - 16s 2ms/step - loss: 0.0529 - acc: 0.9815 - val_loss: 0.2788 - val_acc: 0.9223
Epoch 5/50
10203/10203 [==============================] - 16s 2ms/step - loss: 0.0505 - acc: 0.9805 - val_loss: 0.2680 - val_acc: 0.9324
Epoch 6/50
10203/10203 [==============================] - 16s 2ms/step - loss: 0.0471 - acc: 0.9833 - val_loss: 0.2869 - val_acc: 0.9223
Epoch 7/50
10203/10203 [==============================] - 16s 2ms/step - loss: 0.0456 - acc: 0.9830 - val_loss: 0.3149 - val_acc: 

In [22]:
model2.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-data2_92_acc.h5')

In [ ]:
# as well as adding the pseudo labelling, augment the pseudo labelled data?

In [45]:
# this way will reaugment the orginal images, perhaps doesn't matter? 

Xtrain_pl = get_more_images(Xtrain_pl)
Xinc_pl = np.concatenate((Xinc_pl,Xinc_pl,Xinc_pl))
Ytrain_pl = np.concatenate((Ytrain_pl,Ytrain_pl,Ytrain_pl))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((29289, 75, 75, 3), (29289,), (29289,))

In [46]:
model2.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 29289 samples, validate on 148 samples
Epoch 1/50
29289/29289 [==============================] - 46s 2ms/step - loss: 0.0408 - acc: 0.9864 - val_loss: 0.1343 - val_acc: 0.9561
Epoch 2/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0367 - acc: 0.9876 - val_loss: 0.1561 - val_acc: 0.9459
Epoch 3/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0316 - acc: 0.9882 - val_loss: 0.1464 - val_acc: 0.9662
Epoch 4/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0343 - acc: 0.9881 - val_loss: 0.1358 - val_acc: 0.9595 acc: 0.98 - ETA: 5s - loss: 0.0344 - acc: 0.988 - ETA: 4s - loss: 0.0344 - acc: 0.987 - ETA: 4s - lo
Epoch 5/50
29289/29289 [==============================] - 46s 2ms/step - loss: 0.0291 - acc: 0.9892 - val_loss: 0.1477 - val_acc: 0.9595
Epoch 6/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0299 - acc: 0.9893 - val_loss: 0.1663 - val_acc: 0.9527
Epoch 7/50
29289/29289 [=====

In [65]:
model2.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_model2_96_acc.h5')

In [23]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model2.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])

1471/1471 [==============================] - 1s 636us/step
Train score: 0.0680898964347
Train accuracy: 0.982664853841


In [24]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('/home/ubuntu/data/iceberg/results/final_preds/cnn_model2_pseudo_data2' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  2.231949e-03
1  4023181e  9.414673e-01
2  b20200e4  9.350348e-01
3  e7f018bb  9.999995e-01
4  4371c8c3  1.864136e-01
5  a8d9b1fd  9.991841e-01
6  29e7727e  3.206558e-03
7  92a51ffb  9.999993e-01
8  c769ac97  3.583631e-12
9  aee0547d  8.747134e-11


In [39]:
Xtest.shape, Xtest_inc.shape, pred_test.shape

((8424, 75, 75, 3), (8424,), (8424, 2))

In [40]:
Xtest = get_more_images(Xtest)
Xtest_inc = np.concatenate((Xtest_inc,Xtest_inc,Xtest_inc))
pred_test = np.concatenate((pred_test, pred_test, pred_test))

Xtest.shape, Xtest_inc.shape, pred_test.shape

((25272, 75, 75, 3), (25272,), (25272, 2))

In [41]:
# this is the correct way round with [:,1]
idx_pred_1 = (np.where(pred_test[:,1]>0.95))
idx_pred_0 = (np.where(pred_test[:,1]<0.05))

In [49]:
Xtest_inc = np.expand_dims(Xtest_inc, axis=1)
X_inc_train = np.expand_dims(X_inc_train, axis=1)

In [50]:
X_inc_train.shape, Xtest_inc.shape

((3969, 1), (25272, 1))

In [52]:
X_inc_train = np.squeeze(X_inc_train)
Xtest_inc = np.squeeze(Xtest_inc)
X_inc_train.shape, Xtest_inc.shape

((3969,), (25272,))

In [51]:
Xtrain_pl = np.concatenate((X_train, Xtest[idx_pred_1[0],...], Xtest[idx_pred_0[0],...]))
Ytrain_pl = np.concatenate((Y_train, np.ones(idx_pred_1[0].shape[0]), np.zeros(idx_pred_0[0].shape[0])))
Xinc_pl = np.concatenate((X_inc_train, Xtest_inc[idx_pred_1[0],...], Xtest_inc[idx_pred_0[0],...]))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((22671, 75, 75, 3), (22671,), (22671, 1))

In [53]:
p = 0.3
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p/2)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model3 = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
model3.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=256, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 22671 samples, validate on 148 samples
Epoch 1/50
22671/22671 [==============================] - 34s 2ms/step - loss: 0.5127 - acc: 0.7542 - val_loss: 0.5443 - val_acc: 0.7500
Epoch 2/50
22671/22671 [==============================] - 34s 1ms/step - loss: 0.2778 - acc: 0.8833 - val_loss: 0.5339 - val_acc: 0.7770
Epoch 3/50
22671/22671 [==============================] - 34s 1ms/step - loss: 0.1617 - acc: 0.9312 - val_loss: 0.3537 - val_acc: 0.8851
Epoch 4/50
22671/22671 [==============================] - 34s 1ms/step - loss: 0.1204 - acc: 0.9495 - val_loss: 0.2666 - val_acc: 0.9054
Epoch 5/50
22671/22671 [==============================] - 34s 1ms/step - loss: 0.1006 - acc: 0.9611 - val_loss: 0.2568 - val_acc: 0.9257
Epoch 6/50
22671/22671 [==============================] - 34s 1ms/step - loss: 0.1026 - acc: 0.9620 - val_loss: 0.3428 - val_acc: 0.8885
Epoch 7/50
22671/22671 [==============================] - 34s 1ms/step - loss: 0.0804 - acc: 0.9690 - val_loss: 0.2730 - val_acc: 

In [55]:
model3.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_model3_data2_93_acc.h5')

In [56]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model3.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])

1471/1471 [==============================] - 1s 651us/step
Train score: 0.0615342418751
Train accuracy: 0.986403806934


In [106]:
# submit_nums = np.clip(pred_test[:,1], 0.1, 0.9)
submit_nums[1]

0.89999998

In [57]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('/home/ubuntu/data/iceberg/results/final_preds/cnn_model3_pseudo_aug_data2_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  2.231949e-03
1  4023181e  9.414673e-01
2  b20200e4  9.350348e-01
3  e7f018bb  9.999995e-01
4  4371c8c3  1.864136e-01
5  a8d9b1fd  9.991841e-01
6  29e7727e  3.206558e-03
7  92a51ffb  9.999993e-01
8  c769ac97  3.583631e-12
9  aee0547d  8.747134e-11


In [92]:
df_test.shape

(8424, 4)

In [114]:
model.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)

4413/4413 [==============================] - 3s 600us/step


[0.13649047835023664, 0.95014729196998826]